In [2]:
import pandas as pd
import requests as r

In [3]:
adm = r.get("http://localhost:3000/api/administration")
df = pd.DataFrame(adm.json())
df["parent"] = df["parent"].fillna(0).astype(int)
dict_id = df.set_index('id').parent.to_dict()

def get_parent_id(anc):
    anc = [anc] if not isinstance(anc, list) else anc
    if anc[-1] == 0:
        return anc
    else:
        parent = get_parent_id([dict_id[anc[-1]]])
        anc += parent
        return anc

dict_name = df.set_index('id').name.to_dict()
df['path_id'] = df.id.apply(get_parent_id)
df['parent'] = df.apply(
    lambda x:
    [dict_name[id_] for id_ in x.path_id if not (id_ == x.id or id_ == 0)],
    axis=1)
df['parent'] = df['parent'].apply(lambda x: " | ".join(x))


In [5]:
df[["id","parent","name"]].to_csv("../backend/source/administration-cascade.csv", index=False)